# Optimisation of operation
### Table of Contents

* [1. Introduction](#1.introduction)
* [2. First basic problem](#fbp)
    * [2.1. Math and first step with pyomo for solving the problem](#math)
    * [2.2. Variables ](#variables)
    * [2.3. Constraints](#constraints)
* [3. Extensions of this operation problem ](#temporal)
    * [3.1. Linear temporal coupling with ramp constraints ](#ramp)
    * [3.2. Linear spatial coupling with spatial constraints - Problem Op3 Multi-Area -](#spatial)
* [4. Storage operation ](#storage)
    *[4.1. Optimisation of a storage market participation](#storageMarket)
    *[4.2. Simultaneous optimisation of storage and electric system](#storagecoupling)

## 1. Introduction <a class="anchor" id="1.introduction"></a>
This document contains a description of optimisation tools for electric system operation simulation provided here.

The aim of this part consists in creating a model that enables you to handle the consumption of energy with the lowest prices possible (for the means of production).

In [5]:
import os

myhost = os.uname()[1]
if (myhost=="jupyter-sop"):
    ## for https://jupyter-sop.mines-paristech.fr/ users, you need to
    #  (1) run the following in a terminal
    os.system("/opt/mosek/9.2/tools/platform/linux64x86/bin/lmgrd -c /opt/mosek/9.2/tools/platform/linux64x86/bin/mosek.lic -l lmgrd.log")
    #  (2) definition of license
    os.environ["MOSEKLM_LICENSE_FILE"] = '@jupyter-sop'


if os.path.basename(os.getcwd())=="BasicFunctionalities":
    os.chdir('..') ## to work at project root  like in any IDE

InputFolder='Data/input/'
#region importation of modules
import numpy as np
import pandas as pd
import csv

import datetime
import copy
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn import linear_model
import sys

from functions.f_operationModels import *
from functions.f_optimization import *
from functions.f_graphicalTools import *
#endregion

#region Solver location definition

BaseSolverPath='/Users/robin.girard/Documents/Code/Packages/solvers/ampl_macosx64' ### change this to the folder with knitro ampl ...
## in order to obtain more solver see see https://ampl.com/products/solvers/open-source/
## for eduction this site provides also several professional solvers, that are more efficient than e.g. cbc
sys.path.append(BaseSolverPath)
solvers= ['gurobi','knitro','cbc'] # try 'glpk', 'cplex'
solverpath= {}
for solver in solvers : solverpath[solver]=BaseSolverPath+'/'+solver
solver= 'mosek' ## no need for solverpath with mosek.
#endregion

## 2. First basic problem <a class="anchor" id="fbp"></a>
### 2.1. Math and first step with pyomo for solving problem<a class="anchor" id="math"></a>
\begin{align}
&\text{Cost function }& &\min_{x}  \sum_t \sum_i \pi_i x_{it}\;\;\; & & \pi_i \text{ marginal cost}\\
&\text{Power limit }   & &\text{ s.t.} \;\; 0 \leq x_{it}\leq a_{it} \bar{x_i} & &\bar{x_i} \text{ installed power, }  a_{it} \text{ availability}\\
&\text{Meet demand }   & & \sum_i x_{it} \geq  C_t  && C_t \text{ Consumption}\\
&\text{Stock limit }   & &\sum_t x_{it}\leq E_i && E_i=\bar{x_i}*N_i \text{ Energy capacity limit}\\
\end{align}

If you want to solve the preceeding problem you might have to write everything in a suitable matrix form.
This can be very painful in some cases.
With Pyomo you just need to provide the area consumption, the availability factors and the hypothesis
for the different means of production (in the file Gestion-Simple_TECHNOLOGIES.csv).
Pyomo is then charged of building the matrix form,
and you just have to think about problem formulation.

To build a model, Pyomo needs three different kinds of data :
sets, parameters and variables.

Sets are dimensions, here the time and the name of technology plus a mix of these two :
TIMESTAMP, TECHNOLOGIES and TIMESTAMP_TECHNOLOGIES (product set).

Parameters are tables indexed by set whose values are specified by the user.
Here is the list of the parameters : energycost, EnergyNbhourCap, capacity,
availability factor, area consumption.

variable are tables indexed by set whose values are found by the solver :
the energy produced by each mean of production.

First, we can observe one of the parameters, the availability factor, that has an really important role in the production of electricity

Look at input-XXX.ipynb files to see available input data

In [6]:
#region I - Simple single area : loading parameters
Zones="FR" ; year=2013
#### reading areaConsumption availabilityFactor and TechParameters CSV files
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(Zones)+'.csv',sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',sep=',',decimal='.',skiprows=0)
TechParameters = pd.read_csv(InputFolder+'Gestion-Simple_TECHNOLOGIES.csv',sep=';',decimal=',',skiprows=0)

#### Selection of subset
Selected_TECHNOLOGIES={'OldNuke','Thermal','Solar','WindOnShore'} #you can add technologies here
availabilityFactor=availabilityFactor[ availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters=TechParameters[TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters.loc[TechParameters.TECHNOLOGIES=="WindOnShore",'capacity']=117000
TechParameters.loc[TechParameters.TECHNOLOGIES=="Solar",'capacity']=67000#endregion

Now we run the optimisation script and load optimised variables.

At this stage, or later, you might be willing to to see the pyomo model. If it is the case, dig into function
GetElectricSystemModel_GestionSingleNode (by doing a control-click on it, otherwise it is in functions/f_operationModels.py)

In [7]:
#region I - Simple single area  : Solving and loading results
model = GetElectricSystemModel_GestionSingleNode(areaConsumption,availabilityFactor,TechParameters)
if solver in solverpath :  opt = SolverFactory(solver,executable=solverpath[solver])
else : opt = SolverFactory(solver)
results=opt.solve(model)
## result analysis
Variables=getVariables_panda(model)

#pour avoir la production en KWh de chaque moyen de prod chaque heure
production_df=Variables['energy'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='energy')
production_df.sum(axis=0)/10**6 ### energies produites TWh
print(Variables['energyCosts']) #pour avoir le coût de chaque moyen de prod à l'année
#endregion

  TECHNOLOGIES  energyCosts
0  WindOnShore     0.000000
1        Solar     0.000000
2      OldNuke  1516.900533
3      Thermal   400.388229


We have analysed the result of the optimisation first with the dictionary **variables** that contains all optimized variables.
The most important variable here is 'energy'. The other one 'energyCosts' can be computed from 'energy' and it is somehow redundant.

## 2.2 Analysing results : lagrange multipliers <a class="anchor" id="optiofope"></a>

Lagrange multiplier might be more difficult to understand for those who still lack a good optimisation course (the one at second semester of first year of MINES ParisTech is perfect).
In cases you want to dig this, you can have a look at Boyd's course, e.g. starting with [lecture 8](https://www.youtube.com/watch?v=FJVmflArCXc) or before.
The most important message here is that lagrange multipliers associated to the demand constraint (here called 'energyCtr') are meant to mimic market prices.
Lagrange multiplier associated to this constraint at time t is the marginal cost that one would pay to increase $C_t$ by a small amount. They can be used to simulate market prices.

In [8]:
#region I - Simple single area  : visualisation and lagrange multipliers
### representation des résultats
fig=MyStackedPlotly(x_df=production_df.index,
                    y_df=production_df[list(Selected_TECHNOLOGIES)],
                    Names=list(Selected_TECHNOLOGIES))
fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année")
#plotly.offline.plot(fig, filename='file.html') ## offline
fig.show()

#### lagrange multipliers
Constraints= getConstraintsDual_panda(model)

# Analyse energyCtr
energyCtrDual=Constraints['energyCtr']; energyCtrDual['energyCtr']=energyCtrDual['energyCtr']*1000000
energyCtrDual
round(energyCtrDual.energyCtr,2).unique()

# Analyse CapacityCtr
CapacityCtrDual=Constraints['CapacityCtr'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='CapacityCtr')*1000000;
round(CapacityCtrDual,2)
round(CapacityCtrDual.OldNuke,2).unique() ## if you increase by Delta the installed capacity of nuke you decrease by xxx the cost when nuke is not sufficient
round(CapacityCtrDual.Thermal,2).unique() ## increasing the capacity of Thermal as no effect on prices
#endregion

array([-0.])


We have the **Lagrange multipliers** associated to each (active) **constraint** (zero means unactive constraint) in dictionary 'Constraints'.
We have three different constraints : the energy constraint, the capacity constraint and the storage constraint (unactive here)
Try understanding the meaning of lagrange multipliers, here and in the next Sections.

## 3. Extensions of this operation problem <a class="anchor" id="temporal"></a>
### 3.1.  Linear temporal coupling with ramp constraints <a class="anchor" id="ramp"></a>
We go back to problem Op1 and add : dependency on area z (country), add a congestion constraint, ramp constraints (with rc=0,005)

\begin{align}
&\text{Cost function }& &\min_{x}  \sum_z \sum_t \sum_i \pi_{iz} x_{itz}\;\;\; & & \pi_{iz} \text{ marginal cost}\\
&\text{Power limit }   & &\text{ s.t.} \;\; 0 \leq x_{itz}\leq a_{itz} \bar{x_{iz}} & &\bar{x_{iz}} \text{ installed power, }  a_{itz} \text{ availability}\\
&\text{Meet demand }   & & \sum_i x_{itz} \geq  C_{tz}  && C_{tz} \text{ Consumption}\\
&\text{Stock limit }   & &\sum_t x_{it}\leq E_i && E_i=\bar{x_i}*N_i \text{ Energy capacity limit}\\
&\text{ramp limit }   & &rc^-_i *x_{it}\leq x_{it}-x_{i(t+1)}\leq rc^+_i *x_{it} && rc^+_i rc^-_i\text{ ramp limit}\\
\end{align}


In [9]:
#region II - Ramp Ctrs Single area : loading parameters loading parameterscase with ramp constraints
Zones="FR"
year=2013
Selected_TECHNOLOGIES={'OldNuke','Thermal'} #you'll add 'Solar' after
#### reading CSV files
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
TechParameters = pd.read_csv(InputFolder+'Gestion-RAMP1_TECHNOLOGIES.csv',sep=';',decimal=',',skiprows=0)

#### Selection of subset
availabilityFactor=availabilityFactor[availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters=TechParameters[TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
#endregion

#region II - Ramp Ctrs Single area : solving and loading results
model = GetElectricSystemModel_GestionSingleNode(areaConsumption,availabilityFactor,TechParameters)
opt = SolverFactory(solver)
results=opt.solve(model)
Variables=getVariables_panda(model)

#pour avoir la production en KWh de chaque moyen de prod chaque heure
production_df=Variables['energy'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='energy')
production_df.sum(axis=0)/10**6 ### energies produites TWh
print(Variables['energyCosts']) #pour avoir le coût de chaque moyen de prod à l'année
#endregion

#region II - Ramp Ctrs Single area : visualisation and lagrange multipliers
fig=MyStackedPlotly(x_df=production_df.index,
                    y_df=production_df[list(Selected_TECHNOLOGIES)],
                    Names=list(Selected_TECHNOLOGIES))
fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année")
#plotly.offline.plot(fig, filename='file.html') ## offline
fig.show()

#### lagrange multipliers
Constraints= getConstraintsDual_panda(model)

# Analyse energyCtr
energyCtrDual=Constraints['energyCtr']; energyCtrDual['energyCtr']=energyCtrDual['energyCtr']*1000000
energyCtrDual
round(energyCtrDual.energyCtr,2).unique()

# Analyse CapacityCtr
CapacityCtrDual=Constraints['CapacityCtr'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='CapacityCtr')*1000000;
round(CapacityCtrDual,2)
round(CapacityCtrDual.OldNuke,2).unique() ## if you increase by Delta the installed capacity of nuke you decrease by xxx the cost when nuke is not sufficient
round(CapacityCtrDual.Thermal,2).unique() ## increasing the capacity of Thermal as no effect on prices
#endregion

  TECHNOLOGIES  energyCosts
0      OldNuke  3360.020093
1      Thermal  6857.155133


array([-0.])

Again you have to look at lagrange multipliers. Add renewable production. How does it changes when you add more renewable ?
Try modifying the ramp constraint to generate negative lagrange multipliers.

### 3.2.  Linear spatial coupling with spatial constraints <a class="anchor" id="spatial"></a>

Math here are in 3.1

In [10]:
#region III - Ramp Ctrs multiple area : loading parameters
Zones="FR_DE_GB_ES"
year=2016
Selected_AREAS={"FR","DE"}
Selected_TECHNOLOGIES={'Thermal', 'OldNuke' } #'NewNuke', 'HydroRiver', 'HydroReservoir','WindOnShore', 'WindOffShore', 'Solar', 'Curtailement'}

#### reading CSV files
TechParameters = pd.read_csv(InputFolder+'Gestion_MultiNode_DE-FR_AREAS_TECHNOLOGIES.csv',sep=';',decimal=',',comment="#",skiprows=0)
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)

ExchangeParameters = pd.read_csv(InputFolder+'Hypothese_DE-FR_AREAS_AREAS.csv',sep=';',decimal=',',skiprows=0,comment="#")
#### Selection of subset
TechParameters=TechParameters[TechParameters.AREAS.isin(Selected_AREAS)&TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
areaConsumption=areaConsumption[areaConsumption.AREAS.isin(Selected_AREAS)]
availabilityFactor=availabilityFactor[availabilityFactor.AREAS.isin(Selected_AREAS)& availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
#endregion

#region III - Ramp multiple area : solving and loading results
### small data cleaning
availabilityFactor.availabilityFactor[availabilityFactor.availabilityFactor>1]=1
model = GetElectricSystemModel_GestionMultiNode(areaConsumption,availabilityFactor,TechParameters,ExchangeParameters)
opt = SolverFactory(solver)
results=opt.solve(model)
Variables=getVariables_panda(model)
fig=MyAreaStackedPlot(Variables['energy'])
fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année")
#plotly.offline.plot(fig, filename='file.html') ## offline
fig.shox()

production_df=Variables['energy'].pivot(index=["TIMESTAMP","AREAS"], columns='TECHNOLOGIES', values='energy')
production_df.sum(axis=0)/10**6 ### energies produites TWh
production_df.groupby(by="AREAS").sum()/10**6 ### energies produites TWh

Variables["exchange"].head()
Variables["exchange"].exchange.sum()
Constraints= getConstraintsDual_panda(model)
Constraints.keys()
#endregion

dict_keys(['energyCostsDef', 'exchangeCtr', 'CapacityCtr', 'energyCtr', 'storageCtr', 'rampCtrPlus', 'rampCtrMoins', 'rampCtrPlus2', 'rampCtrMoins2'])

## 4. Storage operation <a class="anchor" id="storage"></a>
### 4.1. Optimisation of a storage market participation <a class="anchor" id="storageMarket"></a>
### 4.2. Simultaneous optimisation of storage and electric system <a class="anchor" id="storagecoupling"></a>

TODO add math here

In [11]:
#region IV Ramp+Storage single area : loading parameters
Zones="FR"
year=2013

Selected_TECHNOLOGIES={'Thermal', 'OldNuke', 'WindOnShore',"Curtailement"}

#### reading CSV files
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
TechParameters = pd.read_csv(InputFolder+'Gestion-Simple_TECHNOLOGIES.csv',sep=';',decimal=',',skiprows=0)

#### Selection of subset
availabilityFactor=availabilityFactor[ availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters=TechParameters[TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]

p_max=10000
StorageParameters={"p_max" : p_max , "c_max": p_max*10,"efficiency_in": 0.9,"efficiency_out" : 0.9}
#endregion

#region IV Ramp+Storage single area : solving and loading results
res= GetElectricSystemModel_GestionSingleNode_with1Storage(areaConsumption,availabilityFactor,
                                                      TechParameters,StorageParameters)

Variables = getVariables_panda(res['model'])
Constraints = getConstraintsDual_panda(res['model'])
areaConsumption = res["areaConsumption"]

production_df=Variables['energy'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='energy')
production_df.loc[:,'Storage'] = areaConsumption["Storage"]### put storage in the production time series
production_df.sum(axis=0)/10**6 ### energies produites TWh
production_df[production_df>0].sum(axis=0)/10**6 ### energies produites TWh
production_df.max(axis=0)/1000 ### Pmax en GW

Selected_TECHNOLOGIES_Sto=list(Selected_TECHNOLOGIES)
Selected_TECHNOLOGIES_Sto.append("Storage")
fig=MyStackedPlotly(x_df=production_df.index,
                    y_df=production_df[Selected_TECHNOLOGIES_Sto],
                    Names=Selected_TECHNOLOGIES_Sto)
fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année")
#plotly.offline.plot(fig, filename='file.html') ## offline
fig.show()
stats=res["stats"]

#endregion

0
1
2
0
1
2


KeyError: 'Storage'

If you want to contribute here, try improving the visualisation tools (with respect to storage, multizone, integration of interconnexions ...)

In [ ]:
#region V Ramp+Storage Multi area : loading parameters
Zones="FR_DE_GB_ES"
year=2016
Selected_AREAS={"FR","DE"}
Selected_TECHNOLOGIES={'Thermal', 'OldNuke' } #'NewNuke', 'HydroRiver', 'HydroReservoir','WindOnShore', 'WindOffShore', 'Solar', 'Curtailement'}

#### reading CSV files
TechParameters = pd.read_csv(InputFolder+'Gestion_MultiNode_DE-FR_AREAS_TECHNOLOGIES.csv',sep=';',decimal=',',comment="#",skiprows=0)
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)

ExchangeParameters = pd.read_csv(InputFolder+'Hypothese_DE-FR_AREAS_AREAS.csv',sep=';',decimal=',',skiprows=0,comment="#")
#### Selection of subset
TechParameters=TechParameters[TechParameters.AREAS.isin(Selected_AREAS)&TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
areaConsumption=areaConsumption[areaConsumption.AREAS.isin(Selected_AREAS)]
availabilityFactor=availabilityFactor[availabilityFactor.AREAS.isin(Selected_AREAS)& availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]

p_max=10000

StorageParameters=pd.DataFrame([])
for AREA in Selected_AREAS :
    StorageParameters_ = {"AREA": AREA, "p_max": p_max, "c_max": p_max * 10, "efficiency_in": 0.9,
                          "efficiency_out": 0.9}
    StorageParameters=StorageParameters.append(pd.DataFrame([StorageParameters_]))

#endregion

#region V Ramp+Storage multi area : solving and loading results
res= GetElectricSystemModel_GestionMultiNode_with1Storage(areaConsumption,availabilityFactor,
                                                      TechParameters,ExchangeParameters,StorageParameters)

Variables = getVariables_panda(res['model'])
Constraints = getConstraintsDual_panda(res['model'])
areaConsumption = res["areaConsumption"]
production_df=Variables['energy'].pivot(index=["AREAS","TIMESTAMP"], columns='TECHNOLOGIES', values='energy')
production_df=pd.concat([production_df,areaConsumption["Storage"]],axis=1)

production_df.sum(axis=0)/10**6 ### energies produites TWh
production_df.groupby(by="AREAS").sum()/10**6 ### energies produites TWh
production_df[production_df>0].groupby(by="AREAS").sum()/10**6 ### energies produites TWh
production_df.max(axis=0)/1000 ### Pmax en GW
#endregion